# How to Develop a Word-Level Neural Language <br> Model and Use it to Generate Text

this part/version A to get next words of seed text.


## Import libs


In [1]:
import spacy
from keras.preprocessing.text import Tokenizer

import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from pickle import dump, load

import random
import pprint

# Prepare Data


In [9]:
def load_doc(path: str) -> str:
    with open(path, 'r') as file:
        data = file.read()
    return data

In [10]:
filepath = '/content/1661-0.txt'
doc = load_doc(filepath)
print(doc[:200])

Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, giv


In [11]:
nlp = spacy.load("en_core_web_sm")


def clean_doc(doc: str):
    text = nlp(doc)
    print('text nlp done.')
    return [i.text.lower() for i in text if i.is_alpha]


toknize = clean_doc(doc)

text nlp done.


In [12]:
print(f'Tokinze len {len(toknize)}')
print(f'Tokinze unique len {len(set(toknize))}')

Tokinze len 107645
Tokinze unique len 8043


In [14]:
prev_words_len = 50
length = prev_words_len + 1

sequences = list()

for i in range(length, len(toknize)):
    sequences.append(' '.join(toknize[i-length:i]))

print(f'the len of sequences: {len(sequences)}')

the len of sequences: 107594


In [15]:
def save_doc(lines, filename):
    with open(filename, 'w') as file:
        file.write('\n'.join(lines))


outlinesSeq = 'r1661-0_sequences.txt'
save_doc(sequences, outlinesSeq)

In [17]:
lines = load_doc('r1661-0_sequences.txt').split('\n')

In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [19]:
vocab_size = len(tokenizer.word_index) + 1

In [20]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_len = X.shape[1]

# Define the Model And Train


In [21]:
model = Sequential()

model.add(Embedding(vocab_size, prev_words_len, input_length=seq_len))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            402200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 8044)              812444    
                                                                 
Total params: 1,365,544
Trainable params: 1,365,544
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
841/841 [==============================] - 62s 62ms/step - loss: 6.4776 - accuracy: 0.0557
Epoch 2/100
841/841 [==============================] - 17s 21ms/step - loss: 6.0066 - accuracy: 0.0757
Epoch 3/100
841/841 [==============================] - 14s 16ms/step - loss: 5.7592 - accuracy: 0.0924
Epoch 4/100
841/841 [==============================] - 13s 16ms/step - loss: 5.5962 - accuracy: 0.1048
Epoch 5/100
841/841 [==============================] - 13s 15ms/step - loss: 5.4648 - accuracy: 0.1161
Epoch 6/100
841/841 [==============================] - 14s 16ms/step - loss: 5.3484 - accuracy: 0.1224
Epoch 7/100
841/841 [==============================] - 12s 14ms/step - loss: 5.2435 - accuracy: 0.1294
Epoch 8/100
841/841 [==============================] - 12s 14ms/step - loss: 5.1474 - accuracy: 0.1347
Epoch 9/100
841/841 [==============================] - 12s 14ms/step - loss: 5.0577 - accuracy: 0.1397
Epoch 10/100
841/841 [==============================] - 13s 15ms/step - l

In [23]:
model.save('nextWordModel.h5')
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Load Model And Use it.


In [25]:
model = load_model('nextWordModel.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))
lines = load_doc('r1661-0_sequences.txt').split('\n')
wordsintok = {i: word for word, i in tokenizer.word_index.items()}

seq_length = len(lines[0].split()) - 1
seq_length

50

## Test for next Word of seed text.


In [27]:
seed_text = random.choice(lines)

pprint.pprint(seed_text, width=50)
encoded = tokenizer.texts_to_sequences([seed_text])[0][:-1]

yhat = model.predict([encoded])

classi = np.argmax(yhat)

print()
pprint.pp(' '.join(seed_text.split(' ')[:-1]))
print(wordsintok[classi])

('without paying any fees or charges if you are '
 'redistributing or providing access to a work '
 'with the phrase project gutenberg associated '
 'with or appearing on the work you must comply '
 'either with the requirements of paragraphs '
 'through or obtain permission for the use of '
 'the work and the project gutenberg')
1/1 [==============================] - 0s 53ms/step

('without paying any fees or charges if you are redistributing or providing '
 'access to a work with the phrase project gutenberg associated with or '
 'appearing on the work you must comply either with the requirements of '
 'paragraphs through or obtain permission for the use of the work and the '
 'project')
gutenberg


## Genrate Sequence of num words After Seed text.


In [28]:
def gen_seq(model, tokenizer, seq_len, seed_text, n_words):
    result = []
    in_text = seed_text
    for _ in range(n_words):
        encodedtxt = tokenizer.texts_to_sequences([in_text])[0]

        encodedtxt = pad_sequences(
            [encodedtxt], maxlen=seq_len, truncating='pre')

        yhat = model.predict([encodedtxt])
        classi = np.argmax(yhat)
        nxtWord = wordsintok[classi]
        in_text += ' ' + nxtWord

        result.append(nxtWord)
    return result

In [29]:
seed_text = random.choice(lines)

gented_txt = gen_seq(model, tokenizer, 50, seed_text, 50)
print('\nThis is Seed Text: ')
pprint.pp(seed_text)

print('\nThis Next 50 Words of Seed Text: ')
pprint.pp(' '.join(gented_txt))

1/1 [==============================] - 0s 32ms/step

This is Seed Text: 
('of it and how have you succeeded well you have a clue i have them in the '
 'hollow of my hand young openshaw shall not long remain unavenged why watson '
 'let us put their own devilish trade mark upon them it is well thought of '
 'what do you mean he took')

This Next 50 Words of Seed Text: 
('a little cry of satisfaction to say the envelope and the creaking of the '
 'countess of morcar the blind that tend to make up the stone in the centre of '
 'the table and wondering lazily who two minor important narrative which led '
 'out into the frosty air remember to be')


## Preidicat the top most num of next words


In [30]:
def next_word_rec(model, tokenizer, seq_len, seed_text, n_words):

    encodedtxt = tokenizer.texts_to_sequences([seed_text])[0]

    encodedtxt = pad_sequences(
        [encodedtxt], maxlen=seq_len, truncating='pre')

    yhat = model.predict([encodedtxt])
    classi = np.argsort(yhat)[0][::-1][:n_words]

    return [wordsintok[wordi] for wordi in classi]

In [31]:
seed_text = random.choice(lines)

top5nextword = next_word_rec(model, tokenizer, 50, seed_text, 5)
print('\nThis is Seed Text: ')
pprint.pp(seed_text)

print(f'\nTop Next 5 Words of Seed Text: {", ".join(top5nextword)}')

1/1 [==============================] - 0s 24ms/step

This is Seed Text: 
('reward offered of is certainly not within a twentieth part of the market '
 'price a thousand pounds great lord of mercy the commissionaire plumped down '
 'into a chair and stared from one to the other of us that is the reward and i '
 'have reason to know that there are sentimental')

Top Next 5 Words of Seed Text: considerations, vague, essential, persuaded, as


# Thanks
